In [2]:
import simpleCongurentGenerators
import time
import numpy as np

In [2]:
sample_size = int(1e6)

In [3]:
## performance check (seconds - ok for educationals purpose)
start = time.time()
M=13
sample_lcg = simpleCongurentGenerators.get_n_lcg(M, 1, 5, 2, size=int(1e6))
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

Elapsed (with compilation) = 2.2192530632019043


In [4]:
sample_lcg

array([ 7, 12,  4, ..., 10,  2,  7], dtype=uint32)

## Map PRNG sample on [0,1] and perform test 

In [4]:
sample_lcg_mapped = sample_lcg / M
assert np.all((sample_lcg_mapped) >= 0 & (sample_lcg_mapped <= 1)) , "not all elements in (0,1)"

In [5]:
from chi_square_test import chi_square_test

In [6]:
test_stat = chi_square_test(sample_lcg_mapped)
test_stat


1420.097041599997

In [7]:
from scipy import stats

In [8]:
stats.kstest(sample_lcg_mapped, 'uniform')

KstestResult(statistic=0.07692346153846152, pvalue=0.0, statistic_location=0.5384615384615384, statistic_sign=1)

## RC(32)

In [15]:
from PRGA import PRGA
from KSA import Keys, KSA

In [17]:
keys = Keys(np.array([3, 4, 2, 5]))
ksa = KSA(m=32)
ksa.schedule(keys)
prga = PRGA(ksa)
start = time.time()
sample_rc = prga.get_random(40)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.0009884834289550781


In [11]:
sample_rc_mapped = sample_rc / ksa.m
assert np.all((sample_rc_mapped) >= 0 & (sample_rc_mapped <= 1)) , "not all elements in (0,1)"

## Frequency monobit

In [12]:
# read data
import pickle

with open('data/binary_expansion_e.pkl', 'rb') as f:
    data = pickle.load(f)

numbers = data["numbers"]
sequence = 2 * numbers - 1 # map into -1 , 1 

## Test function 

In [13]:
import numpy as np
import math
from scipy.stats import norm

def frequency_monobit_test(bits):
#   bits should be already mapped into [-1,1]
    n = len(bits)
    S = np.sum(bits)
    S_obs = abs(S) / math.sqrt(n)
    p_value = 2 * (1 - norm.cdf(S_obs))
    return p_value

In [14]:
p_val = frequency_monobit_test(sequence)
if p_val >= 0.01:
    print("The sequence passes the Frequency (Monobit) Test (p-value: {:.5f})".format(p_val))
else:
    print("The sequence fails the Frequency (Monobit) Test (p-value: {:.5f})".format(p_val))

The sequence passes the Frequency (Monobit) Test (p-value: 0.92846)


In [42]:
def generalized_lcg(m, a, c, x):
    return (np.sum(a * x) + c) % m


def get_n_glcg(m, a, c, init, size):
    assert size >= 1, "Incorrect size"
    depth = len(init)
    result = np.empty(size, dtype=np.uint32)
    result = np.concatenate((init, result))
    for i in range(depth, size+depth):
        result[i] = generalized_lcg(m, a, c, result[i-depth:i])
    return result[-size:]


a = np.array([3, 7, 68])
seed = np.array([1,1,1])
rand = generalized_lcg(2 ** 10, a, 0, seed)
print(rand)

rand_2 = get_n_glcg(2 ** 10, a, 0, seed, 30)

print(rand_2)



78
[  78  194  429   44  433  319  279 1000  254  533   62  517  325  305
 1014  382  197  680  637  539  143   48  762  357   58  537  105  833
  622  314]


In [1]:
np.sum(a*seed)

NameError: name 'np' is not defined

In [12]:
import numpy as np

def is_within_hypercube(vector, lower_bounds, upper_bounds):
    """
    Check if a vector belongs to the hypercube defined by the lower and upper bounds.

    Parameters:
        vector (np.ndarray): The vector in R^m to be checked.
        lower_bounds (np.ndarray): The lower bounds for the hypercube.
        upper_bounds (np.ndarray): The upper bounds for the hypercube.

    Returns:
        bool: True if the vector belongs to the hypercube, False otherwise.
    """
    vector = np.asarray(vector)
    lower_bounds = np.asarray(lower_bounds)
    upper_bounds = np.asarray(upper_bounds)
    
    # Check if the vector is within bounds for all dimensions
    return np.all(vector >= lower_bounds) and np.all(vector <= upper_bounds)

# Example usage:


cube  = np.array([[0, 1],
                 [2, 3],
                  [2, 3]])

vector = np.array([0.5, 2.9, 2.5])  # Vector in R^3
lower_bounds = cube[:, 0]  # Lower bounds for each dimension
upper_bounds = cube[:, 1]  # Upper bounds for each dimension

result = is_within_hypercube(vector, lower_bounds, upper_bounds)
print(f"The vector belongs to the hypercube: {result}")


The vector belongs to the hypercube: True


In [10]:
cube  = np.array([[0, 1],
                 [2, 3],
                  [2, 3]])

lower = cube[:, 0]
upper = cube[:, 1]
lower



array([0, 2, 2])